In [1]:
import scipy.io as sio
import os
import numpy as np
import tensorflow as tf
datapath = r'../data'
dataset = 'CUB' # This is the same CUB data, just 6 classes lacking any DNA barcodes are removed 
x2 = sio.loadmat(os.path.join(datapath, dataset, 'CUB_DNA.mat')) 
x = sio.loadmat(os.path.join(datapath, dataset, 'Bird_DNA.mat'))  # Dataset containing all bird DNA barcodes extracted from COI gene
us_cls = sio.loadmat(os.path.join(datapath, dataset, 'CUB_unseen_classes_sci_name.mat'))
barcodes=x['nucleotides_aligned'][0]
unseen_cls = us_cls['us_classes']
bird_species=x['species'][0]
cub_barcodes = x2['nucleotides_aligned'][0]
c_labels = x2['species'][0]

2022-10-22 15:52:05.550523: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-22 15:52:05.654440: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-22 15:52:06.058344: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-22 15:52:06.058387: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
# Number of training samples and entire data
N = len(barcodes)
print(N)

26409


In [3]:
bcodes=[]
#labels=[]
b_species = []
for i in range(N):
    if len(barcodes[i][0])>0:
        bcodes.append(barcodes[i][0])
        b_species.append(bird_species[i][0])

b_species = np.asarray(b_species)
bcodes = np.asarray(bcodes)
        
# c_species = []
# for i in range(len(cub_species)):
#     c_species.append(cub_species[i][0][0].split('--> ')[1])  
    
cub_bcodes = []
c_labels_ = []
for i in range(len(cub_barcodes)):
    cub_bcodes.append(cub_barcodes[i][0])
    c_labels_.append(c_labels[i][0])
    
unseen_classes = []
for i in range(len(unseen_cls)):
    unseen_classes.append(unseen_cls[i][0][0])

In [4]:
print('Sample barcode: %s' % bcodes[0])

Sample barcode: A-----------------------------------------------------CTATACCTAATCTTCGGCGCCTGAGCTGGTATAGTCGGCACCGCCCTCAGCCTACTCATTCGCGCAGAACTCGGCCAACCAGGCACACTCCTAGGCGACGACCAAATCTACAACGTAATCGTTACTGCACATGCCTTTGTAATAATCTTCTTCATAGTTATACCAATCATGATTGGAGGATTCGGAAACTGACTTGTCCCACTCATAATTGGAGCCCCCGACATAGCCTTTCCACGCATAAACAACATAAGCTTCTGGCTACTCCCTCCATCCTTCCTCCTCCTACTGGCCTCCTCAACAGTAGAAGCAGGAGCCGGCACTGGATGAACTGTCTACCCCCCATTAGCTGGCAACATAGCCCATGCCGGAGCTTCAGTAGACTTGGCCATCTTCTCCTTACACTTAGCTGGAGTCTCATCCATTCTAGGAGCAATCAACTTTATCACAACCGCCATCAACATAAAACCCCCAGCCCTCTCCCAGTACCAAACACCCCTATTCGTATGATCTGTCCTCATTACCGCCGTCCTTCTATTACTTTCACTCCCAGTCCTAGCCGCTGGCATCACTATACTACTCACAGACCGAAACCTAAACACAACATTCTTTGACCCTGCCGGCGGAGGCGATCCTATCCTATACCAACACCTCTTCTGATTCTTTGGACACCCAGAAGTCTACATCTTAATCCT----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
# Excluding unseen classes of CUB data from training set
idx = np.in1d(b_species, unseen_classes)

b_species = b_species[~idx]
bcodes = bcodes[~idx]
print(len(b_species), sum(idx))

25712 697


In [6]:
uy = np.unique(b_species)
labels = np.zeros(len(b_species), dtype='int32')
for i in range(len(uy)):
    idx = b_species==uy[i]
    labels[idx.ravel()] = i+1 # Matlab indexing
    if idx.sum()<10:
        print(uy[i])
    
print('Number of classes: %d' % len(np.unique(labels)))

Number of classes: 999


In [7]:
# Classes and their names from CUB data that has less than 10 samples
# Just for information
uy = np.unique(c_labels_)
c_labels_ = np.asarray(c_labels_)
c_labels = np.zeros(len(c_labels_), dtype='int32')
for i in range(len(uy)):
    idx = c_labels_==uy[i]
    c_labels[idx] = i+1 # Matlab indexing
    if np.sum(idx)<10:
        print(uy[i])
    
print(len(np.unique(c_labels)), np.unique(c_labels))

Aechmophorus occidentalis
Aethia cristatella
Aethia psittacula
Aethia pusilla
Ammodramus bairdii
Ammodramus henslowii
Ammodramus leconteii
Ammodramus maritimus
Ammodramus savannarum
Amphispiza bilineata
Antrostomus carolinensis
Antrostomus vociferus
Aphelocoma coerulescens
Bombycilla cedrorum
Calypte anna
Campylorhynchus brunneicapillus
Cepphus columba
Ceryle rudis
Chordeiles minor
Coccyzus americanus
Coccyzus erythropthalmus
Coccyzus minor
Colibri thalassinus
Contopus cooperi
Corvus albicollis
Corvus ossifragus
Cyanocitta cristata
Cyanocorax yncas
Dolichonyx oryzivorus
Dryocopus pileatus
Empidonax flaviventris
Empidonax minimus
Empidonax virescens
Euphagus carolinus
Fratercula corniculata
Geococcyx californianus
Helmitheros vermivorum
Icteria virens
Icterus cucullatus
Icterus parisorum
Icterus spurius
Lamprotornis nitens
Lanius ludovicianus
Larus californicus
Larus delawarensis
Larus glaucescens
Larus heermanni
Larus occidentalis
Leiothlypis ruficapilla
Leuconotopicus borealis
Leucost

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
# Converting base pairs, ACTG, into numbers: Tokenization

### For BIRD data
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(bcodes)
sequence_of_int = tokenizer.texts_to_sequences(bcodes)

### For CUB data barcodes
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(cub_bcodes)
sequence_of_int_cub = tokenizer.texts_to_sequences(cub_bcodes)

In [9]:
# Bird data barcode lengths have a high variance and not as in INSECT data majority has seq length of 658
# These barcodes are aligned ones as in INSECT data
cnt = 0
ll = np.zeros((len(bcodes), 1))
for i in range(len(bcodes)):
    ll[i] = len(sequence_of_int[i])
    if ll[i]==658:
        #print(i)
        cnt += 1
print(cnt, np.median(ll))

0 1548.0


In [10]:
import numpy as np

# Calculating sequence count for each species
cl = len(np.unique(b_species))
N = len(b_species)
seq_len=np.zeros((cl))
seq_cnt=np.zeros((cl))
for i in range(N):
    k=labels[i]-1
    seq_cnt[k]+=1
    #seq_len[k]+=len(sequence_of_int[k])

In [11]:
print('# classes: %d' % cl)

# classes: 999


In [12]:
# Converting the all CUB DNA barcodes into one-hot encoding. Note that this data is not used during training. 
# We are getting it ready for the prediction time to get the final DNA embeddings after training is done

sl = 1500 # Fixed to the max sequnce length. Note that, changing it to median value, 1548, does have an infitesimal effect
N_cub = len(cub_bcodes)
allX=np.zeros((N_cub,sl,5))
for i in range(N_cub):
    Nt=len(sequence_of_int_cub[i])

    for j in range(sl):
        if(len(sequence_of_int_cub[i])>j):
            k=sequence_of_int_cub[i][j]-1
            if(k>4):
                k=4
            allX[i][j][k]=1.0
            

In [13]:
# Initialize the training matrix and labels
trainX=np.zeros((N,sl,5))
trainY=np.zeros((N,cl))
labelY=np.zeros(N)

In [14]:
# Here is where we cap the class sample size to 50 for a balanced training set

Nc=-1
clas_cnt=np.zeros((cl))
for i in range(N):
    Nt=len(sequence_of_int[i])
    k=labels[i]-1
    clas_cnt[k]+=1
    itl=i+1
    if(seq_cnt[k]>=10 and clas_cnt[k]<=50):
        Nc=Nc+1
        for j in range(sl):
            if(len(sequence_of_int[i])>j):
                k=sequence_of_int[i][j]-1
                if(k>4):
                    k=4
                trainX[Nc][j][k]=1.0
            
        k=labels[i]-1
        trainY[Nc][k]=1.0
        labelY[Nc]=k

In [15]:
print('Final training size: %d' % Nc)

Final training size: 21230


In [16]:
# selecting the balanced training set
trainX=trainX[0:Nc]
trainY=trainY[0:Nc]
labelY=labelY[0:Nc]
print(trainX.shape, trainY.shape, labelY.shape)

(21230, 1500, 5) (21230, 999) (21230,)


In [17]:
idx = np.argwhere(np.all(trainY[..., :] == 0, axis=0))
trainY = np.delete(trainY, idx, axis=1)

print(len(idx), min(np.sum(trainY,axis=0)))

0 10.0


In [18]:
print(trainY.shape)
print(labelY.shape)

(21230, 999)
(21230,)


In [19]:
# Expanding the training set shape for CNN 
trainX=np.expand_dims(trainX, axis=3)
allX=np.expand_dims(allX, axis=3)
print(trainX.shape, allX.shape)

(21230, 1500, 5, 1) (2772, 1500, 5, 1)


In [20]:
from sklearn.model_selection import train_test_split

# Splitting the training set into train and validation
X_train, X_test, y_train, y_test = train_test_split(trainX, trainY, test_size=0.2, random_state=42)

In [21]:
print(y_train.shape)

(16984, 999)


### Classic CNN Model

In [23]:
from tensorflow.keras import datasets, layers, models, optimizers, callbacks

In [24]:
# CNN model architecture
model = models.Sequential()
model.add(layers.Conv2D(64, (3,3), activation='relu', input_shape=(sl, 5,1),padding="SAME"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((3,1)))
model.add(layers.Conv2D(32, (3,3), activation='relu',padding="SAME"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((3,1)))
model.add(layers.Conv2D(16, (3,3), activation='relu',padding="SAME"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((3,1)))
# model.add(layers.Conv2D(16, (3,3), activation='relu',padding="SAME"))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((3,1)))
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(400, activation='tanh'))
model.add(layers.Dense(y_train.shape[1]))

2022-10-22 15:52:33.063740: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-22 15:52:33.069839: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-10-22 15:52:33.069851: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-10-22 15:52:33.070190: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1500, 5, 64)       640       
                                                                 
 batch_normalization (BatchN  (None, 1500, 5, 64)      256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 500, 5, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 500, 5, 32)        18464     
                                                                 
 batch_normalization_1 (Batc  (None, 500, 5, 32)       128       
 hNormalization)                                                 
                                                        

In [26]:
# Step-decay learning rate scheduler
def step_decay(epoch):
   initial_lrate = 0.001
   drop = 0.5
   epochs_drop = 2.0
   lrate = initial_lrate * np.power(drop, np.floor((1+epoch)/epochs_drop))
   return lrate

class LossHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
       self.losses = []
       self.lr = []
 
    def on_epoch_end(self, batch, logs={}):
       self.losses.append(logs.get('loss'))
       self.lr.append(step_decay(len(self.losses)))
        
loss_history = LossHistory()
lrate = callbacks.LearningRateScheduler(step_decay)
callbacks_list = [loss_history, lrate]

In [27]:
from tensorflow.keras.metrics import top_k_categorical_accuracy
#opt = tf.keras.optimizers.SGD(momentum=0.9, nesterov=True)
opt = tf.keras.optimizers.Adam()
model.compile(optimizer=opt, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy','top_k_categorical_accuracy'])
# Validation time
history = model.fit(X_train, y_train, epochs=5, batch_size = 32, validation_data=(X_test, y_test),
                      callbacks=callbacks_list, verbose=1)

# Final Test time
#history = model.fit(trainX, trainY, epochs=5, callbacks=callbacks_list)

Epoch 1/5
531/531 [==============================] - 98s 184ms/step - loss: 1.6566 - accuracy: 0.7677 - top_k_categorical_accuracy: 0.8610 - val_loss: 4.1108 - val_accuracy: 0.3789 - val_top_k_categorical_accuracy: 0.6368 - lr: 0.0010
Epoch 2/5
318/531 [================>.............] - ETA: 38s - loss: 0.1940 - accuracy: 0.9564 - top_k_categorical_accuracy: 0.9960

KeyboardInterrupt: 

In [ ]:
print('Learning rates through epochs:', loss_history.lr)

In [ ]:
### Saving and Loading model
model.save('CUB_DNA_5e_adam_aligned')
model = tf.keras.models.load_model('CUB_DNA_5e_adam_aligned')

In [ ]:
from tensorflow.keras.models import Model
# Getting the DNA embeddings of all data from the last dense layer
layer_name = 'dense'
model2= Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
dna_embeddings=model2.predict(allX)
print(dna_embeddings.shape)

In [ ]:
# np.savetxt("cub_cnn_embeddings_400_5e_adam_aligned.csv", dna_embeddings, delimiter=",")